# Importar os módulos necessários

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
#necessario por causa do metodo de avaliaçao
from sklearn.model_selection import KFold
from sklearn.metrics import (
        make_scorer,
        confusion_matrix, 
        cohen_kappa_score, 
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        roc_auc_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier # decision trees for classification
from sklearn.neural_network import  MLPClassifier # neural networks for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVC # support vector machines for classification

# Definir as Métricas para Avaliação dos Modelos

In [3]:
#metricas regressao
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def custom_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return mse, mae, r2


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import make_scorer
import numpy as np

# Definição da função RMSE
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Dicionário de métricas de regressão
REGRESSION_METRICS = {
    "MSE": make_scorer(mean_squared_error),
    "MAE": make_scorer(mean_absolute_error),
    "R2": make_scorer(r2_score),
    "RMSE": make_scorer(root_mean_squared_error)  # Adicionando o RMSE
}



# Ler o Conjunto de Dados

In [5]:
d = pd.read_csv("Cenario9Old.csv")
d

,id_categoria_marca,kms_driven,resale_price_Lakh
0,0.036241,30910.0,5.66
1,0.036241,48089.0,6.64
2,0.036241,51000.0,5.65
3,0.036241,30000.0,23.00
4,0.036241,61113.0,6.87
...,...,...,...
13897,0.036241,80000.0,26.50
13898,0.036241,17923.0,5.87
13899,0.036241,63389.0,7.43
13900,0.036241,40000.0,9.45


In [6]:
X, y = d.drop("resale_price_Lakh", axis=1), d["resale_price_Lakh"]

# Definir o Método de Validação Cruzada

In [7]:
#Classificação
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

# Utilize KFold para um problema de regressão
kf = KFold(n_splits=10, shuffle=True, random_state=1234)


# Implementação dos Algoritmos de Machine Learning

### Árvores de Decisão

In [8]:
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



# Definir o regressor e o método de validação cruzada
dt_regressor = DecisionTreeRegressor(max_depth=3, random_state=1234)
kf = KFold(n_splits=10, shuffle=True, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores = cross_validate(dt_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
dt_scores = {k: v.mean() for k, v in scores.items()}  # Negar os valores para tornar os scores positivos
pd.DataFrame(dt_scores, index=[0])



,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,0.011291,0.00473,56.915521,4.118194,0.529747,7.506686


### Redes Neuronais

In [9]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_validate

# Criar uma Rede Neural para regressão
nn_regressor = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=20, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_nn = cross_validate(nn_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,1.859957,0.004473,177.632442,7.813798,-0.475077,13.269088


### Naive Bayes

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error



# Criar um regressor RandomForest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=1234)

# Realizar a validação cruzada e avaliar as métricas
scores_rf = cross_validate(rf_regressor, X, y, cv=kf, scoring=REGRESSION_METRICS)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T


,fit_time,score_time,test_MSE,test_MAE,test_R2,test_RMSE
0,4.068233,0.038023,64.703085,4.418299,0.462399,8.013568


Random Forest

### Support Vector Machine